In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

IMG_SIZE = 256 
EPOCHS = 15
dataset_path = './kaggle_3m/kaggle_3m' 

# Busca arquivos
mask_files = glob.glob(os.path.join(dataset_path, '*/*_mask*'))
train_files = [m.replace('_mask', '') for m in mask_files]

if len(train_files) == 0:
    print("ERRO: Nenhuma imagem encontrada.")

df = pd.DataFrame({"image_path": train_files, "mask_path": mask_files})
train_df, val_df = train_test_split(df, test_size=0.15)

# Dataset TensorFlow
AUTOTUNE = tf.data.AUTOTUNE

def read_and_process_image(image_path, mask_path):
    img_p = image_path.decode('utf-8')
    msk_p = mask_path.decode('utf-8')
    
    img = cv2.imread(img_p) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    
    mask = cv2.imread(msk_p, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
    mask = mask / 255.0
    mask = (mask > 0.5).astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    
    return img.astype(np.float32), mask.astype(np.float32)

def tf_process_wrapper(image_path, mask_path):
    img, mask = tf.numpy_function(read_and_process_image, [image_path, mask_path], [tf.float32, tf.float32])
    img.set_shape([IMG_SIZE, IMG_SIZE, 3])
    mask.set_shape([IMG_SIZE, IMG_SIZE, 1])
    return img, mask

def create_dataset_safe(dataframe):
    ds = tf.data.Dataset.from_tensor_slices((dataframe['image_path'].values, dataframe['mask_path'].values))
    ds = ds.map(tf_process_wrapper, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = create_dataset_safe(train_df)
val_ds = create_dataset_safe(val_df)

# Modelo U-Net
from tensorflow.keras import layers, models

def unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = layers.Input(input_size)
    
    # Encoder
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(b)
    u1 = layers.concatenate([u1, c3])
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = layers.concatenate([u2, c2])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    u3 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c5)
    u3 = layers.concatenate([u3, c1])
    c6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u3)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c6)
    return models.Model(inputs=[inputs], outputs=[outputs])

# Métrica Dice
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

print("Iniciando Treinamento...")
# Dica: reduza epochs se seu PC for lento
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS) 
print("Treinamento Concluído.")

# Visualização
def plot_better_prediction(model, dataset):
    for image, mask in dataset.take(1):
        pred_raw = model.predict(image)
        pred_clean = (pred_raw > 0.5).astype(np.float32)
        
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(image[0])
        plt.title("Imagem")
        plt.subplot(1, 3, 2)
        plt.imshow(mask[0], cmap='gray')
        plt.title("Real")
        plt.subplot(1, 3, 3)
        plt.imshow(pred_clean[0], cmap='gray')
        plt.title("Rede")
        plt.show()

plot_better_prediction(model, val_ds)

In [ ]:
# Salva toda a inteligência da rede neural num arquivo físico
model.save('model.keras')